In [1]:
import numpy as np
import pickle
import keras.backend as K

from keras.layers import  GlobalAveragePooling1D, Input, Activation, MaxPooling1D, BatchNormalization, Dense, Dropout, Conv1D,GlobalMaxPooling1D
from keras.layers import GRU,AveragePooling1D,CuDNNGRU
from keras.layers.merge import Concatenate
from keras.models import Model 
from keras.callbacks import EarlyStopping,ModelCheckpoint

import keras.backend.tensorflow_backend as KTF
import tensorflow as tf
import os


os.environ["CUDA_VISIBLE_DEVICES"] = "1"

config = tf.ConfigProto()
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess = tf.Session(config=config)

KTF.set_session(sess)

Using TensorFlow backend.
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/xhh/anaconda3/envs/py36/lib/p

In [2]:
import numpy as np
alphabet = np.array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
                     'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'])

def label_sequence(line, MAX_SEQ_LEN, smi_ch_ind):
	X = np.zeros(MAX_SEQ_LEN)

	for i, ch in enumerate(line[:MAX_SEQ_LEN]):
		X[i] = smi_ch_ind[ch]

	return X #.tolist()

def letter_one_hot(aa):
    one_hot = np.zeros(20)
    for idx, letter in enumerate(alphabet):
        if aa == letter:
            one_hot[idx] = 1
            return one_hot


# Convert an entire protein to one-hot representation.
def protein_one_hot(protein_sequence, MAX_SEQ_LEN):
    #  Remove non-specific AA codes (very few are actually present in this dataset)
    protein_sequence = protein_sequence.replace('B', '')
    protein_sequence = protein_sequence.replace('J', '')
    protein_sequence = protein_sequence.replace('O', '')
    protein_sequence = protein_sequence.replace('U', '')
    protein_sequence = protein_sequence.replace('X', '')
    protein_sequence = protein_sequence.replace('Z', '')
    one_hot_seq = np.zeros( (MAX_SEQ_LEN, 20))
    for idx, aa in enumerate(protein_sequence[:MAX_SEQ_LEN]):
        one_hot_seq[idx, :] = letter_one_hot(aa)
    return one_hot_seq


In [3]:
import keras
feature_len = 768
max_go_len = 128
max_seq_len = 1000

from six.moves import cPickle as pickle #for performance

 
def save_dict(di_, filename_):
    with open(filename_, 'wb') as f:
        pickle.dump(di_, f)

def load_dict(filename_):
    with open(filename_, 'rb') as f:
        ret_di = pickle.load(f)
    return ret_di



class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self,  ppi_pair_file, batch_size=128):
        'Initialization' 
        self.batch_size = batch_size
        self.ppi_pair_file = ppi_pair_file
         
        self.max_seqlen = max_seq_len
        self.max_golen = max_go_len
        self.protein2go =  load_dict('yeast_data/protein2go_dicts.pkl')
         
        self.read_ppi()
        self.prot2emb = {}
        self.prot2embedding()
        self.on_epoch_end()
    
    def read_ppi(self):
        with open(self.ppi_pair_file, 'r') as f:
            self.ppi_pairs  =  f.readlines()
    

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.ppi_pairs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    
    def prot2embedding(self):
        for key, value in self.protein2go.items():
            X_go1 =  np.zeros((1,768))
            allgos = value.split(';') 
            allgos = list(set(allgos))
            count = 0
            for  go in  allgos:
                if go.startswith('GO'):
                    feature = np.load('../ncbi_allfeatures4go/'+go+'_0.npy')[1:-1]
                    if count + feature.shape[0] > max_go_len:
                        break
                    X_go1 = np.concatenate((X_go1,feature ))    
                    count += feature.shape[0]
            self.prot2emb[key] =  X_go1[1:]   
            
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.ppi_pairs))
         
    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization

        X_go1 = np.empty((self.batch_size, self.max_golen,768))
        X_go2 = np.empty((self.batch_size, self.max_golen,768))
        y = np.empty((self.batch_size))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            ppi_pair = self.ppi_pairs[ID]
            p1, p2, label = ppi_pair.rstrip().split('\t')
            if label == '1':
                y[i] = 1
            else:
                y[i] = 0
                
            prot1emb = self.prot2emb[p1]
            X_go1[i,:prot1emb.shape[0]] = prot1emb
            
            prot2emb = self.prot2emb[p2]
            X_go2[i,:prot2emb.shape[0]] = prot2emb
            
#             X_go1[i] =  np.load('SC_GO/'+p1+'.npy') 
#             X_go2[i] =  np.load('SC_GO/'+p2+'.npy')
#             values = self.protein2go[ p1] 
#             allgos = values.split(';') 
#             allgos = list(set(allgos))
#             count = 0
#             for  go in  allgos:
#                 feature = np.load('ncbi_allfeatures4go/'+go+'_0.npy')[1:-1]
#                 if count + feature.shape[0] > self.max_golen:
#                     break
#                 X_go1[i, count:count+feature.shape[0]] = feature
#                 count += feature.shape[0]

#             values = self.protein2go[ p2]
#             allgos = values.split(';') 
#             allgos = list(set(allgos)) 
#             count = 0
#             for  go in  allgos:
#                 feature = np.load('ncbi_allfeatures4go/'+go+'_0.npy')[1:-1]
#                 if count + feature.shape[0] > self.max_golen:
#                     break
#                 X_go2[i, count:count+feature.shape[0]] = feature
#                 count += feature.shape[0]
        return [X_go1,X_go2] ,  y



    def all_data(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization

        X_go1 = np.empty((len(list_IDs_temp), self.max_golen,768))
         
        X_go2 = np.empty((len(list_IDs_temp), self.max_golen,768))
        y = np.empty((len(list_IDs_temp)))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            ppi_pair = self.ppi_pairs[ID]
            p1, p2, label = ppi_pair.rstrip().split('\t')
            if label == '1':
                y[i] = 1
            else:
                y[i] = 0
            
            prot1emb = self.prot2emb[p1]
            X_go1[i,:prot1emb.shape[0]] = prot1emb
            
            prot2emb = self.prot2emb[p2]
            X_go2[i,:prot2emb.shape[0]] = prot2emb
            
#             X_go1[i] =  np.load('SC_GO/'+p1+'.npy') 
#             X_go2[i] =  np.load('SC_GO/'+p2+'.npy')
#             values = self.protein2go[ p1 ]
#             allgos = values.split(';') 
#             allgos = list(set(allgos))
#             count = 0
#             for  go in  allgos:
#                 feature = np.load('ncbi_allfeatures4go/'+go+'_0.npy')[1:-1]
#                 if count + feature.shape[0] > self.max_golen:
#                     break
#                 X_go1[i, count:count+feature.shape[0]] = feature
#                 count += feature.shape[0]

#             values = self.protein2go[ p2]
#             allgos = values.split(';') 
#             allgos = list(set(allgos)) 
#             count = 0
#             for  go in  allgos:
#                 feature = np.load('ncbi_allfeatures4go/'+go+'_0.npy')[1:-1]
#                 if count + feature.shape[0] > self.max_golen:
#                     break
#                 X_go2[i, count:count+feature.shape[0]] = feature
#                 count += feature.shape[0]
        return [X_go1,X_go2] ,  y

In [4]:
from keras import backend as K, initializers, regularizers, constraints
from keras.engine.topology import Layer


def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        # todo: check that this is correct
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)


class Attention(Layer):
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True,
                 return_attention=False,
                 **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Note: The layer has been tested with Keras 1.x
        Example:
            # 1
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
            # next add a Dense layer (for classification/regression) or whatever...
            # 2 - Get the attention scores
            hidden = LSTM(64, return_sequences=True)(words)
            sentence, word_scores = Attention(return_attention=True)(hidden)
        """
        self.supports_masking = True
        self.return_attention = return_attention
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        eij = dot_product(x, self.W)

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        weighted_input = x * K.expand_dims(a)

        result = K.sum(weighted_input, axis=1)

        if self.return_attention:
            return [result, a]
        return result

    def compute_output_shape(self, input_shape):
        if self.return_attention:
            return [(input_shape[0], input_shape[-1]),
                    (input_shape[0], input_shape[1])]
        else:
            return input_shape[0], input_shape[-1]

In [5]:
from keras.layers import   Embedding
from keras.layers import  GRU, Bidirectional, CuDNNGRU, Lambda, dot, Flatten, CuDNNLSTM, Add
from keras.layers.merge import concatenate

def inception_block(input_tensor, output_size):
    """"""
    con1d_filters = int(output_size/4)
    y = Conv1D(con1d_filters, 3, activation="relu", padding='same')(input_tensor)
    x1 = Conv1D(con1d_filters, 5, activation="relu", padding='same')(y)

    y = Conv1D(con1d_filters, 1, activation="relu", padding='valid')(input_tensor)
    x2 = Conv1D(con1d_filters, 3, activation="relu", padding='same')(y)

    x3 = Conv1D(con1d_filters, 3, activation="relu", padding='same')(input_tensor)
    x4 = Conv1D(con1d_filters, 1, activation="relu", padding='same')(input_tensor)

    y = Concatenate()([x1, x2, x3, x4])
#     y = MaxPooling1D(2)(mix0)
    # y = AveragePooling1D()(mix0)
#     y = BatchNormalization()(y)

    return y


def create_share_model():
    con_filters = 128
    X_input = Input(shape=(max_go_len,feature_len))
    x = inception_block(X_input, con_filters)
    flat = Flatten()(x)
    model = Model(X_input, flat)
    return model

 
def build_siamese_model():
    left_input_go = Input(shape=(max_go_len,feature_len))
    right_input_go = Input(shape=(max_go_len,feature_len))
    
    siamese_a = create_share_model()
    siamese_b = create_share_model()
     
    encoded_l = siamese_a(left_input_go)
    encoded_r = siamese_b(right_input_go)
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])

    prediction = Dense(1,activation='sigmoid')(L1_distance)

    siamese_net = Model(inputs=[left_input_go,right_input_go],outputs=prediction)
    
    siamese_net.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return siamese_net



def build_model():
    con_filters = 128
    left_input_go = Input(shape=(max_go_len,feature_len))
    right_input_go = Input(shape=(max_go_len,feature_len))
    
    x = inception_block(left_input_go,con_filters )
    x = Dropout(0.3)(x)
    x_gru = Bidirectional(CuDNNGRU(64, return_sequences=True))(left_input_go)
    x_gru = Dropout(0.3)(x_gru)
    x_a = GlobalAveragePooling1D()(x)
    x_b = GlobalMaxPooling1D()(x)
    x_gru_a = GlobalAveragePooling1D()(x_gru)
    x_gru_b = GlobalMaxPooling1D()(x_gru)
    
    left_x_go = Concatenate()([x_a  , x_b, x_gru_a, x_gru_b])
    
    
    left_x_go = Dense(256, activation='relu')(left_x_go)
     
 
    x = inception_block(right_input_go,con_filters )
    x = Dropout(0.3)(x)
    x_gru =Bidirectional(CuDNNGRU(64, return_sequences=True))(right_input_go)
    x_gru = Dropout(0.3)(x_gru)
    x_a = GlobalAveragePooling1D()(x)
    x_b = GlobalMaxPooling1D()(x)
    x_gru_a = GlobalAveragePooling1D()(x_gru)
    x_gru_b = GlobalMaxPooling1D()(x_gru)
    
    right_x_go = Concatenate()([x_a  , x_b, x_gru_a, x_gru_b])
    right_x_go = Dense(256, activation='relu')(right_x_go)
    
   
   
    x =   Concatenate()([left_x_go  , right_x_go])
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(512, activation='relu')(x)
  
    x = Dense(1)(x)
    output = Activation('sigmoid')(x)
    # model = Model([left_input_go, right_input_go], output)
  
    model = Model([left_input_go, right_input_go], output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = build_model()
model.summary()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 768)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 128, 768)     0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 128, 32)      73760       input_1[0][0]                    
___________________________________________________________________________

In [6]:
from sklearn.model_selection import StratifiedKFold
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
dataset_name = 'SC'
for rep in range(1):
    n_splits = 5
    TPRs =  np.zeros(n_splits)
    FPRs = np.zeros(n_splits)
    Precs = np.zeros(n_splits)
    ACCs = np.zeros(n_splits)
    F1s = np.zeros(n_splits)
    MCCs = np.zeros(n_splits)
    AUCs = np.zeros(n_splits)
    skf = StratifiedKFold(n_splits=n_splits)
    count = 0
    for split in range(n_splits):
        train_pairs_file = 'yeast_data/train'+str(rep)+'-'+str(split)
        test_pairs_file = 'yeast_data/test'+str(rep)+'-'+str(split)
        valid_pairs_file = 'yeast_data/valid'+str(rep)+'-'+str(split)

        batch_size = 256
        train_generator = DataGenerator(   train_pairs_file,batch_size = batch_size )
        test_generator = DataGenerator(   test_pairs_file,batch_size = batch_size)
        valid_generator = DataGenerator(   valid_pairs_file,batch_size = batch_size)
         
        # model = build_model_without_att()
        model = build_model()
        save_model_name = 'yeast_data/sc_go'+str(rep)+'-'+str(split) + '.hdf5'
        
        earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
        save_checkpoint = ModelCheckpoint(save_model_name, save_best_only=True, monitor='val_loss', mode='min', save_weights_only=True)

         
        # validation_data = (valid_X, valid_Y),  verbose=1,callbacks=[earlyStopping, save_checkpoint]
        hist = model.fit_generator(generator=train_generator,
                    validation_data=valid_generator, 
                    epochs = 100,verbose=1,callbacks=[earlyStopping, save_checkpoint] )
         
        
        # model = load_model(save_model_name)
        model.load_weights(save_model_name)
        with open(test_pairs_file, 'r') as f:
            test_ppi_pairs  =  f.readlines()

        test_len = len(test_ppi_pairs) 
        list_IDs_temp = np.arange(test_len)

        test_x, y_test = test_generator.all_data(list_IDs_temp)

        y_pred_prob = model.predict(test_x)

       
        y_pred = (y_pred_prob > 0.5)
        auc = metrics.roc_auc_score(y_test, y_pred_prob) 
        f1 = f1_score(y_test, y_pred)
        pre = precision_score(y_test, y_pred)
        acc = accuracy_score(y_test, y_pred)

        precision, recall, _thresholds = metrics.precision_recall_curve(y_test, y_pred_prob)
        pr_auc = metrics.auc(recall, precision)
        mcc = matthews_corrcoef(y_test, y_pred)

        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        total=tn+fp+fn+tp
        sen = float(tp)/float(tp+fn)
        sps = float(tn)/float((tn+fp))

        tpr = float(tp)/float(tp+fn)
        fpr = float(fp)/float((tn+fp))
        print('--------------------------\n')
        print ('AUC: %f' % auc)
        print ('ACC: %f' % acc) 
        # print("PRAUC: %f" % pr_auc)
        print ('MCC : %f' % mcc)
        # print ('SEN: %f' % sen)
        # print ('SEP: %f' % sps)
        print('TPR:%f'%tpr)
        print('FPR:%f'%fpr)
        print('Pre:%f'%pre)
        print('F1:%f'%f1)
        print('--------------------------\n')
        TPRs[count] = tpr
        FPRs[count] = fpr
        Precs[count] =pre
        ACCs[count] =acc
        F1s[count] =f1
        MCCs[count] =mcc
        AUCs[count] =auc
        count += 1
        del test_x
        del y_test
    print ('mean AUC: %f' % np.mean(AUCs))
    print ('mean ACC: %f' % np.mean(ACCs)) 
    print ('mean MCC : %f' % np.mean(MCCs))
    print('mean TPR:%f'% np.mean(TPRs))
    print('mean FPR:%f'% np.mean(FPRs))
    print('mean Pre:%f'% np.mean(Precs))
    print('mean F1:%f'% np.mean(F1s))
    np.savez('yeast_go_'+str(rep), AUCs=AUCs, ACCs=ACCs, MCCs=MCCs, TPRs = TPRs, FPRs=FPRs, Precs=Precs, F1s=F1s)

Epoch 1/100
27/27 [==============================] - 20s 727ms/step - loss: 0.7534 - acc: 0.6072 - val_loss: 0.6371 - val_acc: 0.6191
Epoch 2/100
27/27 [==============================] - 13s 473ms/step - loss: 0.5916 - acc: 0.6678 - val_loss: 0.5615 - val_acc: 0.7129
Epoch 3/100
27/27 [==============================] - 12s 462ms/step - loss: 0.5300 - acc: 0.7174 - val_loss: 0.5204 - val_acc: 0.7396
Epoch 4/100
27/27 [==============================] - 13s 484ms/step - loss: 0.4860 - acc: 0.7551 - val_loss: 0.5024 - val_acc: 0.7461
Epoch 5/100
27/27 [==============================] - 14s 537ms/step - loss: 0.4439 - acc: 0.7833 - val_loss: 0.5090 - val_acc: 0.7376
Epoch 6/100
27/27 [==============================] - 16s 578ms/step - loss: 0.3909 - acc: 0.8171 - val_loss: 0.4124 - val_acc: 0.8216
Epoch 7/100
27/27 [==============================] - 16s 592ms/step - loss: 0.3473 - acc: 0.8442 - val_loss: 0.3679 - val_acc: 0.8464
Epoch 8/100
27/27 [==============================] - 15s 556ms

27/27 [==============================] - 20s 744ms/step - loss: 0.7576 - acc: 0.6036 - val_loss: 0.6249 - val_acc: 0.6452
Epoch 2/100
27/27 [==============================] - 15s 552ms/step - loss: 0.5751 - acc: 0.6832 - val_loss: 0.5561 - val_acc: 0.7025
Epoch 3/100
27/27 [==============================] - 15s 557ms/step - loss: 0.5174 - acc: 0.7338 - val_loss: 0.5259 - val_acc: 0.7279
Epoch 4/100
27/27 [==============================] - 16s 580ms/step - loss: 0.4707 - acc: 0.7698 - val_loss: 0.5522 - val_acc: 0.7109
Epoch 5/100
27/27 [==============================] - 15s 555ms/step - loss: 0.4353 - acc: 0.7928 - val_loss: 0.4684 - val_acc: 0.7799
Epoch 6/100
27/27 [==============================] - 15s 566ms/step - loss: 0.3768 - acc: 0.8268 - val_loss: 0.4506 - val_acc: 0.7806
Epoch 7/100
27/27 [==============================] - 14s 522ms/step - loss: 0.3350 - acc: 0.8487 - val_loss: 0.4455 - val_acc: 0.8027
Epoch 8/100
27/27 [==============================] - 15s 557ms/step - loss

Epoch 1/100
27/27 [==============================] - 19s 711ms/step - loss: 0.7386 - acc: 0.6043 - val_loss: 0.6234 - val_acc: 0.6341
Epoch 2/100
27/27 [==============================] - 13s 487ms/step - loss: 0.5947 - acc: 0.6727 - val_loss: 0.5998 - val_acc: 0.7012
Epoch 3/100
27/27 [==============================] - 14s 503ms/step - loss: 0.5337 - acc: 0.7253 - val_loss: 0.5116 - val_acc: 0.7311
Epoch 4/100
27/27 [==============================] - 14s 502ms/step - loss: 0.4761 - acc: 0.7688 - val_loss: 0.4653 - val_acc: 0.7708
Epoch 5/100
27/27 [==============================] - 13s 486ms/step - loss: 0.4307 - acc: 0.7976 - val_loss: 0.4673 - val_acc: 0.7676
Epoch 6/100
27/27 [==============================] - 13s 492ms/step - loss: 0.3775 - acc: 0.8270 - val_loss: 0.4113 - val_acc: 0.8236
Epoch 7/100
27/27 [==============================] - 13s 492ms/step - loss: 0.3761 - acc: 0.8241 - val_loss: 0.4595 - val_acc: 0.7767
Epoch 8/100
27/27 [==============================] - 13s 492ms